In [4]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from html_table_parser import parser_functions as parse
from time import sleep
import pandas as pd
import urllib.request
import os
import pprint

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)

### Get a list of all counties in WI

In [107]:
# Get the list of all counties in WI
county_url = "https://dnr.wi.gov/lakes/clmn/"
county_html = requests.get(county_url)

In [108]:
# .text returns the request content in Unicode
county_html.text[:500]

'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n\r\n<head><meta http-equiv="content-type" content="text/html;charset=utf-8" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="icon" /><link href="http://dnr.wi.gov/favicon.ico" type="image/x-icon" rel="shortcut icon" /><title>\r\n\tCitizen Lake Monitoring Network\r\n</title><!-- BEGIN global_head.inc ( /includes/global_head'

In [109]:
county_soup = bs(county_html.text, 'html.parser')

In [110]:
county_soup.find_all(name='li', attrs={'class':'multiColList'})

[<li class="multiColList"><a href="Stations.aspx?location=1">
                 Adams County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=2">
                 Ashland County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=3">
                 Barron County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=4">
                 Bayfield County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=5">
                 Brown County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=6">
                 Buffalo County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=7">
                 Burnett County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=8">
                 Calumet County
             </a></li>,
 <li class="multiColList"><a href="Stations.aspx?location=9"

In [111]:
# Extract list of all counties, correctly formatted 
counties = []
for entry in county_soup.find_all(name='li', attrs={'class':'multiColList'}):
    counties.append(entry.text.replace('\r\n', '').replace('                ','').replace('            ','').\
                    replace(' County','').replace('Fond du Lac','Fond Du Lac').replace('Saint Croix','St. Croix')) 
    
counties

['Adams',
 'Ashland',
 'Barron',
 'Bayfield',
 'Brown',
 'Buffalo',
 'Burnett',
 'Calumet',
 'Chippewa',
 'Clark',
 'Columbia',
 'Crawford',
 'Dane',
 'Dodge',
 'Door',
 'Douglas',
 'Dunn',
 'Eau Claire',
 'Florence',
 'Fond Du Lac',
 'Forest',
 'Grant',
 'Green',
 'Green Lake',
 'Iowa',
 'Iron',
 'Jackson',
 'Jefferson',
 'Juneau',
 'Kenosha',
 'Kewaunee',
 'La Crosse',
 'Lafayette',
 'Langlade',
 'Lincoln',
 'Manitowoc',
 'Marathon',
 'Marinette',
 'Marquette',
 'Menominee',
 'Milwaukee',
 'Monroe',
 'Oconto',
 'Oneida',
 'Outagamie',
 'Ozaukee',
 'Pepin',
 'Pierce',
 'Polk',
 'Portage',
 'Price',
 'Racine',
 'Richland',
 'Rock',
 'Rusk',
 'St. Croix',
 'Sauk',
 'Sawyer',
 'Shawano',
 'Sheboygan',
 'Taylor',
 'Trempealeau',
 'Vernon',
 'Vilas',
 'Walworth',
 'Washburn',
 'Washington',
 'Waukesha',
 'Waupaca',
 'Waushara',
 'Winnebago',
 'Wood']

### Get all reports containing CHLA readings

Get all station numbers

In [112]:
# find station number in from table column station ID 
# All counties: https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=0 
# By county: https://dnr.wi.gov/lakes/clmn/Stations.aspx?location=1 


stationIDs = []

def stationID_extract(pageNumber, url):
    driver = webdriver.Firefox()
    station_url = url
    driver.get(station_url)
    
    for i in range(pageNumber):
        station = driver.page_source
        station_soup = bs(station, "html.parser")
        station_table = station_soup.find('table',  { "class" : "greysuitsyou" })
        station_twod_array = parse.make2d(station_table)
        for n in range(2,len(station_twod_array)-1):
            stationIDs.append(station_twod_array[n][1])
        #sleep(1)
        # go to next page
        driver.find_element_by_id("ctl00_ctl00_LeftPageContent_gvStationTable_ctl01_LinkButton3").click()
    return stationIDs
    driver.close()
    


In [113]:
stationID_extract(40)

['504001',
 '163120',
 '643042',
 '433364',
 '643401',
 '10051075',
 '433354',
 '033182',
 '643121',
 '10051069',
 '663050',
 '443187',
 '493122',
 '443121',
 '143311',
 '143122',
 '083044',
 '433300',
 '073063',
 '493106',
 '673226',
 '073123',
 '033171',
 '10051124',
 '10051076',
 '643125',
 '643126',
 '443399',
 '443060',
 '10051010',
 '493126',
 '493114',
 '433252',
 '433081',
 '363064',
 '10051079',
 '643055',
 '363036',
 '363120',
 '433366',
 '353095',
 '583092',
 '443075',
 '513098',
 '583204',
 '643177',
 '683371',
 '10044915',
 '643057',
 '203079',
 '073086',
 '653215',
 '10031264',
 '10031263',
 '443089',
 '263044',
 '443096',
 '073077',
 '073081',
 '203076',
 '683122',
 '143123',
 '363035',
 '263124',
 '10051077',
 '10051078',
 '10049242',
 '263138',
 '593044',
 '433337',
 '643066',
 '643434',
 '043093',
 '074013',
 '363068',
 '643196',
 '653289',
 '443111',
 '133419',
 '693107',
 '443113',
 '433078',
 '073054',
 '013178',
 '363309',
 '203059',
 '193006',
 '583046',
 '513087

In [120]:
len(stationIDs)

1600

Report url example: stationNo, year1=1950&year2=2017 (regardless of most recent data)
http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=013159&year1=1950&year2=2017&format=csv
http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=10021087&year1=1950&year2=2017&format=csv
http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=013178&year1=1950&year2=2017&format=csv
http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=273120&year1=1950&year2=2017&format=csv

Detail page examples:
https://dnr.wi.gov/lakes/CLMN/Station.aspx?id=013159

In [2]:
def clean_reports(number_of_stations):
    """
    This module takes reports from each station and returns cleaned reports
    
    """
    # get the reports
    for i in range(number_of_stations):
        try:
            report_url = 'http://dnrx.wisconsin.gov/swims/public/reporting.do?type=58&action=post&stationNo=' + str(stationIDs[i]) + '&year1=1950&year2=2017&format=csv'
            #print(report_url)
            original_report_name = './data/wi-lakes/wi_lakes_reports_05_22_2018/report_'+str(stationIDs[i])+'.csv'

            if os.path.isfile(original_report_name):
                continue
            else:
                urllib.request.urlretrieve(report_url, original_report_name)  

            # clean report 
            f = open(original_report_name,'r')
            lines = f.readlines()
            f.close()
            goodlines = []
            start = 0
            for n in range(len(lines)):
                if "Chlorophyll" in lines[n]:
                    start = n
                if start > 0:
                    if lines[n].strip().replace(",","") == "":
                        goodlines = lines[start:n]
                        break


            # get the water body ID which corresponds with satellite paths 
            wbic = lines[4].split(',')[2]

            # get rid of unnecessary columns 
            cleanlines = []
            for l in range(len(goodlines)):
                cleanlines.append(",".join(goodlines[l].split(',')[:10])+'\n')

            #print("\n".join(cleanlines))

            # save clean report 
            clean_report_name = './data/wi-lakes/report_'+str(stationIDs[i])+'_clean.csv'   
            f = open(clean_report_name,'w')      
            f.writelines(cleanlines)
            f.close()
            
        except:
            print('There is an issue with Station with ID ' + str(stationIDs[i]))

    

In [115]:
clean_reports(len(stationIDs))

In [121]:
wi_lakes = []

def reports_to_df(number_of_stations):
    for i in range(number_of_stations):
        try:
            clean_report_name = './data/wi-lakes/wi_lakes_reports_05_22_2018/report_'+str(stationIDs[i])+'_clean.csv'   
            original_report_name = './data/wi-lakes/wi_lakes_reports_05_22_2018/report_'+str(stationIDs[i])+'.csv'

            f = open(original_report_name,'r')
            lines = f.readlines()
            wbic = str(lines[4].split(',')[2])
            lake_name = lines[4].split(',')[0]
            county = lines[4].split(',')[1]

            df = pd.read_csv(clean_report_name,sep=",")
            # clean column names
            df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '_').str.replace(')', '').str.replace('/', '_').str.replace('?', '')
            df['wbic'] = wbic
            df['lake_name'] = lake_name
            df['county'] = county
            df['station_id'] = stationIDs[i]

            wi_lakes.append(df)
        except: 
            print('There is no report for Station with ID ' + str(stationIDs[i]))

    #wi_lakes_df = pd.concat(wi_lakes, ignore_index = True)
    #return wi_lakes_df
    #wi_lakes_df.to_csv('./data/wi-lakes/wi_lakes_all.csv',index=False)
        

In [122]:
reports_to_df(len(stationIDs))

There is no report for Station with ID 10051124
There is no report for Station with ID 143123
There is no report for Station with ID 683388
There is no report for Station with ID 443453
There is no report for Station with ID 10051125
There is no report for Station with ID 113079
There is no report for Station with ID 693029
There is no report for Station with ID 643107


In [13]:
wi_lakes_insitu_df = pd.concat(wi_lakes, ignore_index = True)

In [14]:
wi_lakes_insitu_df

,group_seq_no,start_date,secchi__feet,secchi_hit_bottom,secchi__meters,chlorophyll_ug_l,total_phosphorus_ug_l,secchi_tsi,total_phosphorus_tsi,chlorophyll_tsi,wbic,lake_name,county,station_id
0,1,08/07/1979,NaN,NaN,NaN,3.04,11.0,NaN,47.0,43.0,267800,Adams Lake,Portage,504001
1,81853857,06/19/2013,25.00,NO,7.60,NaN,NaN,31.0,NaN,NaN,267800,Adams Lake,Portage,504001
2,1,07/17/2017,NaN,NaN,NaN,3.38,26.3,NaN,53.0,44.0,267800,Adams Lake,Portage,504001
3,18138029,07/17/2017,6.00,NO,1.80,NaN,NaN,51.0,NaN,NaN,267800,Adams Lake,Portage,504001
4,18138029,08/07/2017,7.75,NO,2.40,NaN,NaN,48.0,NaN,NaN,267800,Adams Lake,Portage,504001
5,1,08/08/2017,NaN,NaN,NaN,2.99,22.8,NaN,52.0,43.0,267800,Adams Lake,Portage,504001
6,18138029,09/13/2017,7.75,NO,2.40,5.88,26.2,48.0,53.0,48.0,267800,Adams Lake,Portage,504001
7,1,08/16/1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,967400,Aldridge Lake,Oneida,443052
8,7000068,08/29/2001,NaN,NaN,NaN,5.80,NaN,NaN,NaN,48.0,967400,Aldridge Lake,Oneida,443052
9,8188599,08/29/2001,3.00,NO,0.90,NaN,NaN,61.0,NaN,NaN,967400,Aldridge Lake,Oneida,443052


### Obtain the coordinates and other information by water body IDs


Obtain information from each water body fact page: https://dnr.wi.gov/lakes/lakepages/LakeDetail.aspx?wbic=267800&page=facts

In [2]:
def get_unique_items(df, col, item_list = []):
    '''
    Get a list of unique items from a dataframe column
    df: dataframe name
    col: column name (str)
    '''
    unique_item = df[col].drop_duplicates()
    item_count = unique_item.count()

    for i in unique_item.index:
        item_list.append(df[col][i])

    return item_list

In [4]:
wi_lakes_insitu_df = pd.read_pickle('./data/wi-lakes/wi_lakes_all.pkl')

In [5]:
wbic = []
get_unique_items(wi_lakes_insitu_df, 'wbic', wbic)

['267800',
 '967400',
 '967900',
 '245650',
 '2128100',
 '968100',
 '2359700',
 '268200',
 '2268600',
 '2858100',
 '968500',
 '458700',
 '2668600',
 '2953800',
 '1441100',
 '968800',
 '2624200',
 '181400',
 '417400',
 '1374300',
 '1377700',
 '854300',
 '2410400',
 '2092500',
 '1626400',
 '417500',
 '2340700',
 '2112800',
 '2620600',
 '2382300',
 '2881200',
 '828600',
 '2662400',
 '969700',
 '2901100',
 '417900',
 '1832800',
 '1580300',
 '2450900',
 '2451200',
 '2451900',
 '1604200',
 '969600',
 '2450800',
 '462400',
 '521400',
 '2450500',
 '2451300',
 '5551281',
 '2718500',
 '552100',
 '1527800',
 '471200',
 '2105100',
 '279700',
 '418000',
 '2618100',
 '1523600',
 '262300',
 '182000',
 '835100',
 '2081200',
 '2081500',
 '77300',
 '894700',
 '2678300',
 '2327100',
 '418300',
 '1545600',
 '2453300',
 '1405200',
 '2705700',
 '2627000',
 '2641000',
 '971600',
 '25300',
 '2244200',
 '2453400',
 '1835100',
 '1427400',
 '1610700',
 '182100',
 '2334700',
 '1613000',
 '345100',
 '2615900',
 '2

In [6]:
len(wbic)

1049

In [7]:
wbic_url = 'https://dnr.wi.gov/lakes/lakepages/LakeDetail.aspx?wbic='+'181400'+'&page=facts'
wbic_html = requests.get(wbic_url)
wbic_soup = bs(wbic_html.text, "html.parser")
wbic_table = wbic_soup.find('table',  { "class" : "tableLeft" })
wbic_twod_array = parse.make2d(wbic_table)
wbic_twod_array

[['Facts & Figures', 'Facts & Figures'],
 ['Name', 'Arbutus Lake'],
 ['Waterbody ID (WBIC)', '181400'],
 ['Area', '163\r\n                    ACRES'],
 ['Maximum Depth', '28 feet'],
 ['Mean Depth', '12 FEET'],
 ['Bottom',
  '65% sand,\r\n                    15% gravel,\r\n                    5% rock,\r\n                    15% muck'],
 ['Waterbody Type', 'lake'],
 ['Hydrologic Lake Type', 'SEEPAGE'],
 ['County', 'Forest'],
 ['Region', 'NO'],
 ['Latitude, Longitude', '45.40135090,\r\n                    -88.86134240'],
 ['Contour (Bathymetric) Map', 'Contour Map'],
 ['Interactive Map', 'Interactive Map'],
 ['Lake Management', 'Lake Management'],
 ['Lake Organizations', 'Arbutus Lake Association Inc [exit DNR]'],
 ['Recreation', 'Recreation'],
 ['Boat Landings', '1'],
 ['Fish', 'Panfish, Largemouth Bass and Walleye'],
 ['Lake Health', 'Lake Health'],
 ['Invasive Species *',
  'Banded Mystery Snail, Chinese Mystery Snail, Freshwater Jellyfish'],
 ['Trophic Status', 'Mesotrophic']]

In [8]:
coordinates = []
depth_max_ft = []
depth_mean_ft = []
trophic_status = []
waterbody_type = []
area_acre = []


wbic_depth_max  = []
wbic_depth_mean = []
wbic_tropic = []
wbic_coord = []
wbic_waterbody = []
wbic_area = []
wbic_failed = []


# GEE takes Lon, Lat

def lake_info_extract():
    '''
    extract information about lakes
    '''
    
    for i in range(len(wbic)): 
        try:

            wbic_url = 'https://dnr.wi.gov/lakes/lakepages/LakeDetail.aspx?wbic='+wbic[i]+'&page=facts'
            wbic_html = requests.get(wbic_url)
            wbic_soup = bs(wbic_html.text, "html.parser")
            wbic_table = wbic_soup.find('table',  { "class" : "tableLeft" })
            wbic_twod_array = parse.make2d(wbic_table)

            for row in wbic_twod_array:

                if 'Area' in row[0]:
                    area_acre.append(row[1].lower().replace('\r\n                    acres',''))
                    wbic_area.append(wbic[i])
                else:
                    pass


                if 'Maximum Depth' in row[0]:
                    depth_max_ft.append(row[1].lower().replace(' feet',''))
                    wbic_depth_max.append(wbic[i])
                else:
                    pass


                if 'Mean Depth' in row[0]:
                    depth_mean_ft.append(row[1].lower().replace(' feet',''))
                    wbic_depth_mean.append(wbic[i])
                else:
                    pass

                if 'Waterbody Type' in row[0]:
                    waterbody_type.append(row[1].lower())
                    wbic_waterbody.append(wbic[i])
                else:
                    pass


                if 'Trophic Status' in row[0]:
                    trophic_status.append(row[1].lower())
                    wbic_tropic.append(wbic[i])
                else:
                    pass


                if 'Latitude, Longitude' in row[0]:
                    coord = []
                    coord_string = row[1].replace('\r\n                    ','').split(',')
                    coord.append(float(coord_string[1]))
                    coord.append(float(coord_string[0]))
                    coordinates.append(coord)
                    wbic_coord.append(wbic[i])

                else:
                    continue

                    
        except:
            print('There is an issue with water body ' + str(wbic[i]))
            wbic_failed.append(wbic[i])

        

In [9]:
lake_info_extract()

There is an issue with water body 5551281
There is an issue with water body 2693500
There is an issue with water body 130300
There is an issue with water body 1246800
There is an issue with water body 1179900
There is an issue with water body 805000
There is an issue with water body 
There is an issue with water body 5535807
There is an issue with water body 2317100
There is an issue with water body 2689800
There is an issue with water body 1352800
There is an issue with water body 440200
There is an issue with water body 868400
There is an issue with water body 2659400
There is an issue with water body 2310500
There is an issue with water body 5582235
There is an issue with water body 5588789
There is an issue with water body 5577031
There is an issue with water body 5566558


In [10]:
# these water bodies have no detail page
len(wbic_failed)

19

In [14]:
len(coordinates)

1030

In [11]:
depth_max_dict = dict(zip(wbic_depth_max, depth_max_ft))
depth_mean_dict = dict(zip(wbic_depth_mean, depth_mean_ft))
coord_dict = dict(zip(wbic_coord, coordinates))
tropic_dict = dict(zip(wbic_tropic, trophic_status))
waterbody_dict = dict(zip(wbic_waterbody, waterbody_type))
area_dict = dict(zip(wbic_area, area_acre))

In [12]:
wi_lakes_insitu_df['coordinates'] = wi_lakes_insitu_df['wbic'].map(coord_dict)
wi_lakes_insitu_df['depth_max_ft'] = wi_lakes_insitu_df['wbic'].map(depth_max_dict)
wi_lakes_insitu_df['depth_mean_ft'] = wi_lakes_insitu_df['wbic'].map(depth_mean_dict)
wi_lakes_insitu_df['tropic_status'] = wi_lakes_insitu_df['wbic'].map(tropic_dict)
wi_lakes_insitu_df['waterbody_type'] = wi_lakes_insitu_df['wbic'].map(waterbody_dict)
wi_lakes_insitu_df['area_acre'] = wi_lakes_insitu_df['wbic'].map(area_dict)

In [13]:
wi_lakes_insitu_df

,group_seq_no,start_date,secchi__feet,secchi_hit_bottom,secchi__meters,chlorophyll_ug_l,total_phosphorus_ug_l,secchi_tsi,total_phosphorus_tsi,chlorophyll_tsi,wbic,lake_name,county,station_id,coordinates,depth_max_ft,depth_mean_ft,tropic_status,waterbody_type,area_acre
0,1,08/07/1979,NaN,NaN,NaN,3.04,11.0,NaN,47.0,43.0,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
1,81853857,06/19/2013,25.00,NO,7.60,NaN,NaN,31.0,NaN,NaN,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
2,1,07/17/2017,NaN,NaN,NaN,3.38,26.3,NaN,53.0,44.0,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
3,18138029,07/17/2017,6.00,NO,1.80,NaN,NaN,51.0,NaN,NaN,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
4,18138029,08/07/2017,7.75,NO,2.40,NaN,NaN,48.0,NaN,NaN,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
5,1,08/08/2017,NaN,NaN,NaN,2.99,22.8,NaN,52.0,43.0,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
6,18138029,09/13/2017,7.75,NO,2.40,5.88,26.2,48.0,53.0,48.0,267800,Adams Lake,Portage,504001,"[-89.3813373, 44.4366549]",44,17,mesotrophic,lake,30
7,1,08/16/1979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,967400,Aldridge Lake,Oneida,443052,"[-89.3012762, 45.8659415]",12,NaN,eutrophic,lake,142
8,7000068,08/29/2001,NaN,NaN,NaN,5.80,NaN,NaN,NaN,48.0,967400,Aldridge Lake,Oneida,443052,"[-89.3012762, 45.8659415]",12,NaN,eutrophic,lake,142
9,8188599,08/29/2001,3.00,NO,0.90,NaN,NaN,61.0,NaN,NaN,967400,Aldridge Lake,Oneida,443052,"[-89.3012762, 45.8659415]",12,NaN,eutrophic,lake,142


In [16]:
wi_lakes_insitu_df.to_pickle('./data/wi-lakes/wi_lakes_all.pkl') 
wi_lakes_insitu_df.to_csv('./data/wi-lakes/wi_lakes_all.csv',index=False) 